In [1]:
# from tensorflow.keras.datasets import mnist
# from imutils import build_montages
import numpy as np
# import cv2

import pandas as pd
import re
import os
import numpy as np

from torch.utils.data import Dataset, DataLoader
import torch
import torchvision
import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import warnings
import math
from numpy import random
from torch.nn.functional import softmax



from torch.autograd import Variable
from torch.optim import *

from pathlib import Path

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import random
from google.colab import drive

warnings.filterwarnings("ignore")
%matplotlib inline

torch.cuda.empty_cache()

In [2]:
crop = 'soy'
mdl = 'pixss_3dcnn_msa_nfc_smp'
# debug_state = 26

In [3]:
test_years = [2019]
tst_year = test_years[0]
start_year = 2014
train_years = list (range(start_year,tst_year))
val_years = [tst_year-2, tst_year-1]

print('train_years',train_years)
print('val_years',val_years)
print('test_years',test_years)
len(train_years), len(test_years), tst_year, crop, mdl

train_years [2014, 2015, 2016, 2017, 2018]
val_years [2017, 2018]
test_years [2019]


(5, 1, 2019, 'soy', 'pixss_3dcnn_msa_nfc_smp')

In [4]:
if crop == 'soy':
    timesteps=list (range(16,43)) #soybean
#     timesteps=list (range(1,47)) #soybean
else:
    timesteps=list (range(8,43)) #corn
#     timesteps=list (range(0,46)) #soybean

len(timesteps)

27

In [5]:
# GPU related info
cuda = 1
device = torch.device("cuda:0" if torch.cuda.is_available() and cuda == 1 else "cpu") # default gpu
print("Device:", device)

Device: cuda:0


In [6]:
# # random pixel selecter in every epoch


# from numpy.random import seed
# from numpy.random import randint

# MAX_HEIGHT = 400
# MAX_WIDTH = 400

# height_indices = []
# weight_indices = []

# seed(1)

# def update_random_indices(arr_size=400):
#     global height_indices, weight_indices
#     height_indices = randint(0,MAX_HEIGHT, arr_size)
#     weight_indices= randint(0,MAX_WIDTH, arr_size)

In [7]:
rc = torchvision.transforms.Resize((150, 250),interpolation=T.InterpolationMode.NEAREST) ##original 150,250


In [8]:
class CustomImageDataset(Dataset):
    def __init__(self, dir_path, file_names, startyear,no_samples):
        self.img_dir = dir_path
        self.img_list = file_names
        self.no_samples = no_samples
        self.startyear = startyear


    def __len__(self):
        return self.no_samples*len(self.img_list)

    def __getitem__(self, idx):

        file_num = idx//self.no_samples

        file_name = self.img_list[file_num]
        sample_num = idx%self.no_samples + self.startyear
#         print(idx,file_num,sample_num)
#         print(file_name)



        img_path = os.path.join(self.img_dir, file_name)
        files = np.load(img_path, allow_pickle=True)
        states   = torch.tensor(files[files.files[0]])
        counties = torch.tensor(files[files.files[1]])
        years    = torch.tensor(files[files.files[2]])

        red_img     = rc(torch.tensor(files[files.files[3]]))
        blue_img     = rc(torch.tensor(files[files.files[4]]))
        green_img     = rc(torch.tensor(files[files.files[5]]))
        nir_img     = rc(torch.tensor(files[files.files[6]]))
        swir_img     = rc(torch.tensor(files[files.files[7]]))
        yld_img      = torch.tensor(files[files.files[8]])



        img_state  = states[0].numpy()
        img_county = counties[0].numpy()
        img_state  = np.asarray(img_state, dtype = 'int')
        img_county = np.asarray(img_county, dtype = 'int')

        red_img   = red_img[:,timesteps,:,:]
        blue_img  = blue_img[:,timesteps,:,:]
        green_img = green_img[:,timesteps,:,:]
        nir_img   = nir_img[:,timesteps,:,:]
        swir_img  = swir_img[:,timesteps,:,:]

        red_img   = red_img[sample_num]
        blue_img  = blue_img[sample_num]
        green_img = green_img[sample_num]
        nir_img   = nir_img[sample_num]
        swir_img  = swir_img[sample_num]


        states   = torch.unsqueeze(states[0], 0)
        counties = torch.unsqueeze(counties[0], 0)
        years    = years[sample_num]
        yld_img  = torch.unsqueeze(yld_img[0], 0)



#         df_mets = df_mets[sample_num]

        year  = years.numpy()
        year  = np.asarray(year, dtype = 'int')



        sfbands = torch.stack((red_img, blue_img, green_img,nir_img,swir_img), 0)
#         return_value =  [eviimgs,msrimgs,psriimgs,srimgs,wdrviimgs,saviimgs,ndwiimgs,gliimgs,ndyiimgs,ndviimgs,ndviylds]
#         print(ndviimgs.shape, ndvistates.shape, ndviyear)
        return_value =  [sfbands,yld_img]
        # print(return_value.shape)
        return return_value

In [9]:
files_count = 0
# folder path
# dir_path = r'../../corn_npz/'
#dir_path = r'../aryan_mayank/modis_soy_npzsample'
# dir_path = r'../../data_sample_5/'
# dir_path = r'./data_sample/'
# drive.mount('/content/drive')
# dir_path = r'/content/drive/My Drive/Research_work/Problems/img_handling_patches/data_sample'

drive.mount('/content/gdrive')
dir_path = r'gdrive/My Drive/data_sample'
#print(os.listdir(filename))

dir_list = os.listdir(dir_path)
print(dir_list)

# df_mets, ndvistates, ndvicounties, ndviimgs, ndwiimgs, saviimgs, ndyiimgs, psriimgs, ndviylds
dataset = CustomImageDataset(dir_path,dir_list,0,5)

#print(dataset.__getitem__(0))
item = dataset[0]
fnames = ['sf_channels',  'train_yld']
print([(fnames[i],item[i].shape) for i in range(len(item))])




Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
['5_1.npz', '5_3.npz', '5_17.npz', '5_21.npz', '5_31.npz']
[('sf_channels', torch.Size([5, 27, 150, 250])), ('train_yld', torch.Size([1]))]


In [10]:
start_idx = 0
train_data = CustomImageDataset(dir_path,dir_list,start_idx,len(train_years))
vlidx = start_idx+len(train_years)-2
tstidx = start_idx+len(train_years)
val_data = CustomImageDataset(dir_path,dir_list,vlidx,2)
test_data = CustomImageDataset(dir_path,dir_list,tstidx,1)

In [11]:
batchSize = 16
train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=16, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size=8, shuffle=False)

In [12]:
#pix_num = 5

class pix_sel(nn.Module):
    def __init__(self):
        super(pix_sel,self).__init__()



    def forward(self,item,pix_num):

        cur_image_height = item.shape[3]
        cur_image_width = item.shape[4]


        x_idx = random.sample(range(0,cur_image_height), pix_num)
        y_idx = random.sample(range(0,cur_image_width), pix_num)

        return x_idx,y_idx

In [13]:
patch_size = 13
rc1 = torchvision.transforms.Resize((patch_size, patch_size),interpolation=T.InterpolationMode.NEAREST)
class patch_sel(nn.Module):
    def __init__(self):
        super(patch_sel,self).__init__()
        self.pix = pix_sel()


    def forward(self,item,pix_num,x,y,cnt,flag):

        cur_image_height = item.shape[3]
        cur_image_width = item.shape[4]

        cnt_res = []
        x_res = []
        y_res = []
        patches = []
        if flag == False:
          x_idx,y_idx = self.pix(item,pix_num)
          cnt = []
          for i in range(len(x_idx)):
              x_idxm = x_idx[i]+patch_size
              y_idxm = y_idx[i]+patch_size
              if x_idxm > cur_image_height:
                  x_end = cur_image_height
                  x_st  = x_idx[i]-(x_idxm - cur_image_height)
              else:
                  x_end = x_idxm
                  x_st  = x_idx[i]

              if y_idxm > cur_image_width:
                  y_end = cur_image_width
                  y_st  = y_idx[i]-(y_idxm - cur_image_width)
              else:
                  y_end = y_idxm
                  y_st  = y_idx[i]

              imx = item[:,:,:,x_st:x_end,:]
              imp = imx[:,:,:,:,y_st:y_end]
              patches.append(imp)
              cnt.append(1)
          for i in range (len(item)):
            x_res.append(x_idx)
            y_res.append(y_idx)
            cnt_res.append(cnt)
        else:
          for i in range(pix_num):
            patches.append([])
          for j in range(len(item)):
            x_idx = x[j]
            y_idx = y[j]
            cnt_idx = cnt[j]
            curr_item = item[j]
            for i in range(len(x_idx)):
              x_idxm = x_idx[i]+patch_size*cnt_idx[i]
              y_idxm = y_idx[i]+patch_size*cnt_idx[i]
              if x_idxm > cur_image_height:
                  x_end = cur_image_height
                  x_st  = x_idx[i]-(x_idxm - cur_image_height)
              else:
                  x_end = x_idxm
                  x_st  = x_idx[i]

              if y_idxm > cur_image_width:
                  y_end = cur_image_width
                  y_st  = y_idx[i]-(y_idxm - cur_image_width)
              else:
                  y_end = y_idxm
                  y_st  = y_idx[i]
              imx = curr_item[:,:,x_st:x_end,:]
              imp = imx[:,:,:,y_st:y_end]
              imp = rc1(imp)
              patches[i].append(imp)
          x_res = x
          y_res = y
          cnt_res = cnt
          for i in range(len(x[0])):
            patches[i] = torch.stack(patches[i])
        return patches,x_res,y_res,cnt_res

In [14]:
X_range = 15
Y_range = 25
def initializeGrid(batch):
  grid = []
  for k in range (0,batch):
    temp = []
    for i in range (0,X_range):
      for j in range (0,Y_range):
        temp.append([i,j])
    grid.append(temp)
  return grid


In [34]:
def getRandGrid(item,batch):
  index = random.randint(0,len(grid[batch])-1)
  res = grid[batch][index]
  grid[batch].pop(index)
  return getPatch(item,res,batch)

In [35]:
# Need to call this per batch item
def getPatch(item,index,batch):
  i = index[0]
  j = index[1]
  x_st = i*10
  x_end = (i+1)*10
  y_st = j*10
  y_end = (j+1)*10
  item = item[batch]
  imx = item[:,:,x_st:x_end,:]
  imp = imx[:,:,:,y_st:y_end]
  print(imp.shape)
  return imp

In [17]:
def getNeighbouringPatch(item,index):
  x = index[0]
  y = index[1]
  res = []
  for i in range (x-1,x+1):
    for j in range (y-1,y+1):
      if([i,j] in grid):
        res.append(getPatch(item,[i,j]))
        grid.remove([i,j])
  return res

In [18]:
def attention(q, k, v, d_k, mask=None, dropout=None):

    scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)

    scores = F.softmax(scores, dim=-1)



    output = torch.matmul(scores, v)
    return output

In [19]:
def softmax_calc(x):
    max = torch.max(x)
    sub = torch.sub(x,max)
    e_x = torch.exp(sub)
    return e_x / torch.sum(x,axis=0)

#res = softmax(np.array([[1,2,3,6]]))
#res[0][3]

In [20]:
num_patches = 5
def highest_att(y,vectors,curr_x,curr_y,cnt):
  res = []
  top_x = []
  top_y = []
  top_cnt = []
  # for i in range(0,num_patches):
  #   res.append([])
  for j in range(0,vectors[0].size(dim=0)):
    top_x.append([])
    top_y.append([])
    top_cnt.append([])
    res.append([])
    x = y[j]
    indices = torch.topk(x,num_patches).indices
    num = 0;
    for (i,idx) in enumerate(indices):
      top_x[j].append(curr_x[j][idx])
      top_y[j].append(curr_y[j][idx])
      top_cnt[j].append(cnt[j][idx]+1)
      res[j].append(vectors[i][idx])
      num = num+1
  for i in range(0,vectors[0].size(dim=0)):
    res[i] = torch.stack(res[i])
  return res,top_x,top_y,top_cnt

In [21]:
#Number of patches
def attention_calc(vectors,w_q,w_k,w_v,n):

  new_patch = []
  new_vectors = []
  for i in range(0,vectors[0].size(dim=0)):
    new_vectors.append([])
  for i in range(0,n):
    for j in range(0,vectors[i].size(dim=0)):
      new_vectors[j].append(vectors[i][j])


  ans = []
  for i in range(0,vectors[0].size(dim=0)):
    new_patch = new_vectors[i]
    new_patch = torch.stack(new_patch)
    new_patch = new_patch.to(device)
    w_q = w_q.to(device)
    w_k = w_k.to(device)
    w_v = w_v.to(device)
    Q = new_patch@w_q
    K = new_patch@w_k
    V = new_patch@w_v
    Q.to(device)
    K.to(device)
    V.to(device)
    attn_scores = Q @ K.T
    attn_scores_softmax = softmax(attn_scores, dim=-1)
    attn_scores_softmax = torch.tensor(attn_scores_softmax)
    weighted_values = V[:,None] * attn_scores_softmax.T[:,:,None]
    outputs = weighted_values.sum(dim=0)
    fin = []
    for i in range(0,n):
      fin.append(torch.sum(outputs[i]))
    fin = torch.stack(fin)
    res = softmax_calc(fin)
    ans.append(res)
  ans = torch.stack(ans)
  return ans



In [22]:
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout = 0.1):
        super().__init__()

        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads

        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, mask=None):

        bs = q.size(0)

        # perform linear operation and split into h heads

        k = k.to(device)
        q = q.to(device)
        v = v.to(device)
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)

        # transpose to get dimensions bs * h * sl * d_model

        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)

        # matmul

        mm = torch.matmul(q,k.transpose(2,3))
        # print('matmul', mm.shape)

        mm1 = nn.Softmax(dim=1)

        sf = mm1(mm/math.sqrt(d_model))
        # print('sf', type(sf))

        mm2 = torch.matmul(sf,v)

        scores = attention(q, k, v, self.d_k)
        # print('scores',scores.shape)

        # scores_reshaped = scores.permute(3,1,2,0)
        # print('scores_reshaped',scores_reshaped.shape)

        concat = scores.transpose(1,2).contiguous()\
        .view(bs, -1, self.d_model)
        # print('concat', concat.shape)
        output = self.out(concat)
        print('output_ttt', output.shape)
        return output

In [23]:

class cnn3d(nn.Module):


    def __init__(self, landsat_inc,ps,sl,dp):
        super(cnn3d, self).__init__() # 46,150,135
        self.conv1 = nn.Conv3d(in_channels = landsat_inc, out_channels = 10, kernel_size =3, stride=1, padding = 0) #  46,150,135
        self.pool = nn.MaxPool3d(kernel_size =3, stride =1, padding =0) # 46,120,105
        self.conv2 = nn.Conv3d(in_channels = 10, out_channels = 15, kernel_size =3, stride=1, padding = 0) #46,90,75
        self.conv3 = nn.Conv3d(in_channels = 15, out_channels = 20, kernel_size =3, stride=1, padding = 0) #46,90,75
#         self.pool = nn.MaxPool2d(kernel_size =(2,1), stride =1)
        #self.conv3 = nn.Conv3d(in_channels = 3, out_channels = 4, kernel_size =(1,11,21), stride=1, padding = 0) #46,60,45
        self.dropout = nn.Dropout(dp)
        self.bn1 = nn.BatchNorm3d(10)
        self.bn2 = nn.BatchNorm3d(15)
        self.bn3 = nn.BatchNorm3d(20)
        self.fc1 = nn.Linear((20*(ps-12)*(ps-12)*(sl-12)), 2000) #4000
        self.fc2 = nn.Linear(2000, 200) #2000
#         self.fc3 = nn.Linear(1000, enc_out_dim) #2000
        self.layernorm_1 = nn.LayerNorm(2000, elementwise_affine = False)
        self.layernorm_2 = nn.LayerNorm(200,  elementwise_affine = False)

    def forward(self,x):

        x = F.relu(self.conv1(x))
#         print('conv1',x.shape)
        x = self.pool(x)
#         print('maxpool1', x.shape)
        x = self.bn1(x)
#         print('bn1', x.shape)

        x = F.relu(self.conv2(x))
#         print('conv2', x.shape)
        x = self.pool(x)
#         print('maxpool2',x.shape)
        x = self.bn2(x)
#         print('bn2', x.shape)

        x = F.relu(self.conv3(x)) # reduced the layers, due to time constraints.
#         print('conv3',x.shape)
        x = self.pool(x)
#         print('maxpool3',x.shape)
        x = self.bn3(x)
#         print('bn3', x.shape)

        x = self.dropout(x)
        nsamples, nc,nt, nx, ny = x.shape
        out = x.reshape((nsamples,nc*nt*nx*ny))

#         print('out_reshape', out.shape)
        ll_1 = self.fc1(out)
# #         print('ll_1 shape', ll_1.shape)
        sigmoid = nn.ReLU()
        out = sigmoid(ll_1)
        out = self.layernorm_1(out)
#         print('out ln1', out.shape)

        ll_2 = self.fc2(out)
#         print('ll_2 shape', ll_2.shape)
        relu = nn.ReLU()
        out = relu(ll_2)
        out = self.layernorm_2(out)
        # print('out ln2', out.shape)

#         x = self.fc3(out)
#         #dim3 = int(enc_out_dim/nt) ## doubt no use

# #         print('x shape', dim3)
#         #x = x.reshape(x.shape[0],nt,dim3) ## doubt no use

        return out



In [37]:
loop_cnt = 10
nxt_iter_patch = 2
patch_cnt = 7
np.random.seed(100)
W_Q = torch.empty(200,patch_cnt)
torch.nn.init.xavier_normal_(W_Q)
W_K = torch.empty(200,patch_cnt)
torch.nn.init.xavier_normal_(W_K)
W_V = torch.empty(200,patch_cnt)
torch.nn.init.xavier_normal_(W_V)
grid = initializeGrid(batchSize)
# W_Q1 = torch.empty(1000,loop_cnt)
# torch.nn.init.xavier_normal_(W_Q1)
# W_K1 = torch.empty(1000,loop_cnt)
# torch.nn.init.xavier_normal_(W_K1)
# W_V1 = torch.empty(1000,loop_cnt)
# torch.nn.init.xavier_normal_(W_V1)
class prediction_model(nn.Module):

	def __init__(self, landsat_inc,ps,hidden_dim,sl,dp,num_heads,out_dim):
		super(prediction_model, self).__init__()

		self.cnn3d = cnn3d(landsat_inc,ps,sl,dp)
		self.patch = patch_sel()
		self.linear_layer1 = nn.Linear(hidden_dim, 1000)
# 		self.linear_layer2 = nn.Linear(1000, 500)
# 		self.linear_layer3 = nn.Linear(500, 250)
# 		self.linear_layer4 = nn.Linear(250, 100)
		self.output_layer = nn.Linear(1000, out_dim) #MHASoil
# 		self.output_layer = nn.Linear(hidden_dim, output_dim)
		self.attn = MultiHeadAttention(num_heads, hidden_dim)
		self.layernorm = nn.LayerNorm([1000], elementwise_affine = False) #MHASoil

	def forward(self,X):
		intvect = []
		top_x = []
		top_y = []
		top_cnt = []
		grid = initializeGrid(batchSize)
		for i in range(loop_cnt):
			intvect1 = torch.zeros([patch_cnt, X.shape[0],200], dtype=torch.float)
			patch_out = []
			curr_x = []
			curr_y = []
			cnt = []
			if i == 0:
				for j in range (0,patch_cnt):
					temp = []
					for k in range (0,batchSize):
						temp.append(getRandGrid(X,k))
					temp = torch.stack(temp)
					patch_out.append(temp)
			print(len(patch_out))
			print(patch_out[0].shape)

			# else:
			# 	patch_out1,curr_x1,curr_y1,cnt1 = self.patch(X,nxt_iter_patch,top_x,top_y,top_cnt,False)
			# 	patch_out2,curr_x2,curr_y2,cnt2 = self.patch(X,num_patches,top_x,top_y,top_cnt,True)

			# 	for j in range(len(patch_out1)):
			# 		patch_out.append(patch_out1[j])

			# 	for j in range(len(patch_out2)):
			# 		patch_out.append(patch_out2[j])
			# 	for j in range(len(curr_x1)):
			# 		curr_x.append([])
			# 		curr_y.append([])
			# 		cnt.append([])
			# 		for k in range(len(curr_x1[0])):
			# 			curr_x[j].append(curr_x1[j][k])
			# 			curr_y[j].append(curr_y1[j][k])
			# 			cnt[j].append(cnt1[j][k])
			# 		for k in range(len(curr_x2[0])):
			# 			curr_x[j].append(curr_x2[j][k])
			# 			curr_y[j].append(curr_y2[j][k])
			# 			cnt[j].append(cnt2[j][k])

			for j in range(len(patch_out)):
				outcnn = self.cnn3d(patch_out[j])
				intvect1[j] = outcnn
			attention_val = attention_calc(intvect1,W_Q,W_K,W_V,patch_cnt)
			attention_val = attention_val.to(device)
			intvect1,top_x,top_y,top_cnt = highest_att(attention_val,intvect1,curr_x,curr_y,cnt)
			for j in range (len(intvect1)):
				intvect1[j] = torch.flatten(intvect1[j])
			intvect1 = torch.stack(intvect1)
			intvect.append(intvect1)
		intvect = torch.stack(intvect)
		intvec = intvect.permute(1,0,2)

		nsamples, nx, ny = intvec.shape
		llinp = torch.reshape(intvec,(nsamples,nx*ny))
		llinp = llinp.to(device)
		outmsa = self.attn(llinp,llinp,llinp,torch.Tensor([1,1])) #MHA
		outmsa = outmsa.to(device)

		l1 = self.linear_layer1(outmsa)
		sigmoid = nn.Sigmoid()
		outl1 = sigmoid(l1)
		outln = self.layernorm(outl1)
		out = self.output_layer(outln)
		return out


In [25]:
# path_to_save = '../../saved_models/'+mdl +'_'+str(tst_year)
path_to_save = 'gdrive/My Drive/data_sample_results/'
path_to_save

# path_to_output = '../../../saved_outputs/landsat8_corn_2020'

'gdrive/My Drive/data_sample_results/'

In [26]:
def evaluate_vims(model,data_loader, dataset):
    model.eval()

    eval_loss1 = float(0)
    eval_loss2 = float(0)

    correct = 0
    avg_loss = []
    output_list = []
    target_list = []
    count = 0
    with torch.no_grad():
        for data, target in data_loader:
            data, target =  (data.float()).to(device),target.float().to(device)

#             sf_features, sf_features_aug = cl_model(data_sf)
#             loss_1 = cont_loss_itr(sf_features,sf_features_aug)

            output = model(data)#.view(-1)
#             print('output',output.shape)
#             print('target',target.shape)
#             target = target.repeat(1,output.shape[1])
            output = output.squeeze(-1)
#             print('output1',output.shape)
#             print('target1',target.shape)
            eval_lossb1 = criterion_print(output, target).mean(axis=0)*len(target)
            eval_lossb2 = torch.sqrt(criterion_print(output, target)).mean(axis=0)* len(target)
            eval_lossb1 = torch.tensor(eval_lossb1)
            eval_lossb2 = torch.tensor(eval_lossb2)
            eval_loss1 += eval_lossb1
#             print('eval_loss1', eval_loss1.shape)
#             print('eval_lossb1', eval_lossb1.shape)
            eval_loss2 += eval_lossb2
            count += len(target)
            output_list += output.tolist()
            target_list += target.tolist()

    eval_loss1 /= count
    eval_loss1 = torch.sqrt(torch.tensor(eval_loss1))
    eval_loss2 /= count
    eval_loss1 = eval_loss1.cpu().numpy()
    eval_loss2 = eval_loss2.cpu().numpy()
#     eval_loss2 = np.mean(avg_loss)
    print('{} set: loss1: {}, loss2: {}\n'.format(dataset, eval_loss1[-1], eval_loss2[-1]))
    if dataset == 'Validation':
      return eval_loss1,eval_loss2, output_list, target_list
    if dataset == 'Test':
      return eval_loss1,eval_loss2, output_list, target_list


In [27]:
def train_vims(model, optimizer, epcs):

    mse_loss = []

    history_train = []
    history_val1 = []
    history_val2 = []

    output_train = []
    target_train = []

    for epoch in range(1, epcs+1):
#         update_random_indices()
#         train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
#         val_dataloader = DataLoader(val_data, batch_size=32, shuffle=False)
        # test_dataloader = DataLoader(test_data, batch_size=32, shuffle=False)
        model.train()
        for batch_idx, (data,target) in enumerate(train_dataloader):

            # data_evi,data_msri,data_psri,data_sri,data_wdrvi,data_savi,data_gli,data_ndwi,data_ndyi,data_ndvi,data_met,data_s, target = (data_evi.float()).to(device), (data_msri.float()).to(device), (data_psri.float()).to(device), (data_sri.float()).to(device), (data_wdrvi.float()).to(device), (data_savi.float()).to(device),(data_gli.float()).to(device), (data_ndwi.float()).to(device), (data_ndyi.float()).to(device), (data_ndvi.float()).to(device), (data_met.float()).to(device), (data_s.float()).to(device),target.float().to(device)
            data ,target  = (data.float()).to(device), target.float().to(device)
#             print('data',data.shape)

#             print('target',target.shape)
            output = model(data)#.view(-1)
#             print('output',output.shape)

#             target = target.repeat(1,output.shape[1])
            output = output.squeeze(-1)
#             print('target1',target.shape)
#             print('output1',output.shape)
            loss_3 = torch.sqrt(criterion_pred(output,target))

            loss = loss_3 # + loss_4 + loss_1 + loss_2
            loss.backward()
            optimizer.step()

        if (epoch>2):
            torch.save(model, os.path.join(path_to_save, "model_soy_%d.pt"%(epoch)))
        print('Train Epoch: {} Training Loss: {}'.format(epoch,loss.item()))

        result_val1,result_val2,train_preds, labels = evaluate_vims(model,val_dataloader,'Validation')

        mse_loss.append(loss_3.item())

        history_train.append(loss.item())
        history_val1.append(result_val1)
        history_val2.append(result_val2)

        output_train.append(train_preds)
        target_train.append(labels)



    return history_train, history_val1,history_val2,output_train, target_train, mse_loss # ,cl_loss1, cl_loss2,cls_loss,

In [28]:
def init_weights(module):
    if type(module) == nn.Linear:
        torch.nn.init.xavier_uniform_(module.weight)
        # torch.nn.init.xavier_normal_(module.weight, gain = 1.0)
        module.bias.data.fill_(0.001)
    if type(module) == nn.Conv3d:
        torch.nn.init.xavier_uniform(module.weight)
        module.bias.data.fill_(0.001)

In [29]:
landsat_inc = 5

seq_length = len(timesteps)
hidden_dim = 10000
epochs = 3   # 75
n_layers = 1
output_dim = 1
ptch_size = patch_size

num_heads = 4
d_model = hidden_dim
dp = 0.25

In [38]:
# timesteps, hidden_dim,sl, out_dim

model_vims = prediction_model(landsat_inc,ptch_size,hidden_dim,seq_length,dp,num_heads,output_dim).to(device)
model_vims.apply(init_weights)
criterion_pred = nn.MSELoss().to(device)
criterion_print = nn.MSELoss(reduce = False, reduction = 'None').to(device)
optimizer = optim.Adam(model_vims.parameters(), lr=0.00001)
# optimizer = optim.SGD(model_vims.parameters(), lr=0.0001, weight_decay = 0.00001, momentum = 0.3)

In [39]:
history_t, history_v1, history_v2, output_train, target_train,  ms_l = train_vims(model_vims, optimizer, epochs) # cl_l2,cl_l1, cl_l2,

torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 10, 10])
torch.Size([5, 27, 1

RuntimeError: ignored

In [ ]:
# # plt.figure(figsize=(8,6))
# # plt.title('cl Loss vs Epochs')
# # plt.plot([x for x in cl_l1])

# # plt.figure(figsize=(8,6))
# # plt.title('cl Loss vs Epochs')
# # plt.plot([x for x in cl_l2])

plt.figure(figsize=(8,6))
plt.title('mse Loss vs Epochs')
plt.plot([x for x in ms_l])


plt.figure(figsize=(8,6))
plt.title('Training Loss vs Epochs')
plt.plot([x for x in history_t])

plt.figure(figsize=(8,6))
plt.title('Validation Loss vs Epochs')
plt.plot([x for x in history_v2])

In [ ]:

# losses1 = []
# losses2 = []

# epochs = 27

# mdls = []
# for m in range(15,epochs+1):
#     model_trained=torch.load(os.path.join(path_to_save, "model_soy_%d.pt"%(m)))
#     loss1, loss2, preds, labels = evaluate_vims(model_trained, test_dataloader, 'Test')
#     losses1.append(loss1[-1])
#     losses2.append(loss2[-1])

#     mdls.append(m)
# print( min(losses1), min(losses2) )

In [ ]:
# df_test = pd.DataFrame(test_counties, columns=['State','County','Year'])
# df_test['Yield'] = test_yld
# df_test['Atual'] = labels
# df_test['Predicted'] = preds
# df_test[0:12]

In [ ]:
# best = mdls[losses2.index(min(losses2))]
# best

In [ ]:
# model_trained=torch.load(os.path.join(path_to_save, "model_soy_%d.pt"%(best)))
# loss1, loss2, preds, labels = evaluate_vims(model_trained, test_dataloader, 'Test')
# loss_met_0 = loss2[-1]
# # loss_met_1 = loss2[-2]
# # loss_met_2 = loss2[-3]
# # loss_met_3 = loss2[-4]
# # loss_met_4 = loss2[-5]
# # loss_met_5 = loss2[-6]
# # loss_met_6 = loss2[-7]
# # loss_met_7 = loss2[-8]
# # loss_met_8 = loss2[-9]
# # loss_met_9 = loss2[-10]
# print('loss_met_0', loss_met_0)
# # print('loss_met_1', loss_met_1)
# # print('loss_met_2', loss_met_2)
# # print('loss_met_3', loss_met_3)
# # print('loss_met_4', loss_met_4)
# # print('loss_met_5', loss_met_5)
# # print('loss_met_6', loss_met_6)
# # print('loss_met_7', loss_met_7)
# # print('loss_met_8', loss_met_8)
# # print('loss_met_9', loss_met_9)

In [ ]:
# fin_label = np.array(labels)
# fin_label = fin_label[:,-1]
# fin_preds = np.array(preds)
# fin_preds = fin_preds[:,-1]

In [ ]:
# df_test = pd.DataFrame(test_states, columns=['State'])
# df_test['County'] = test_counties
# df_test['Year']   = tst_year
# df_test['Yield'] = test_yld
# df_test['Atual'] = fin_label
# df_test['Predicted'] = fin_preds
# df_test[0:12]

In [ ]:
# # import sklearn
# from sklearn import metrics
# from sklearn.metrics import r2_score
# rsquare_vis = r2_score(labels,preds)
# mae_vis = metrics.mean_absolute_error(labels,preds)

In [ ]:
# rmse_df = pd.DataFrame({'loss_1': min(losses1), 'loss_2': min(losses2), 'r_square':rsquare_vis,'mae':mae_vis}, index = ['lr_00001_'+crop+'_'+mdl+'_'+str(tst_year)])
# rmse_df

In [ ]:
# rmse_df_ep= pd.DataFrame({'loss_2': loss_met_0}, index = ['rmse2_0_lr_00001_'+crop+'_'+mdl+'_'+str(tst_year)])
# # rmse_df_1 = pd.DataFrame({'loss_2': loss_met_1}, index = ['rmse2_1'+crop+'_'+mdl+'_'+str(tst_year)])
# # rmse_df_2 = pd.DataFrame({'loss_2': loss_met_2}, index = ['rmse2_2'+crop+'_'+mdl+'_'+str(tst_year)])
# # rmse_df_3 = pd.DataFrame({'loss_2': loss_met_3}, index = ['rmse2_3'+crop+'_'+mdl+'_'+str(tst_year)])
# # rmse_df_4 = pd.DataFrame({'loss_2': loss_met_4}, index = ['rmse2_4'+crop+'_'+mdl+'_'+str(tst_year)])
# # rmse_df_5 = pd.DataFrame({'loss_2': loss_met_5}, index = ['rmse2_5'+crop+'_'+mdl+'_'+str(tst_year)])
# # rmse_df_6 = pd.DataFrame({'loss_2': loss_met_6}, index = ['rmse2_6'+crop+'_'+mdl+'_'+str(tst_year)])
# # rmse_df_7 = pd.DataFrame({'loss_2': loss_met_7}, index = ['rmse2_7'+crop+'_'+mdl+'_'+str(tst_year)])
# # rmse_df_8 = pd.DataFrame({'loss_2': loss_met_9}, index = ['rmse2_8'+crop+'_'+mdl+'_'+str(tst_year)])







# # rmse_df_ep = rmse_df_ep.append(rmse_df_1)
# # rmse_df_ep = rmse_df_ep.append(rmse_df_2)
# # rmse_df_ep = rmse_df_ep.append(rmse_df_3)
# # rmse_df_ep = rmse_df_ep.append(rmse_df_4)
# # rmse_df_ep = rmse_df_ep.append(rmse_df_5)
# # rmse_df_ep = rmse_df_ep.append(rmse_df_6)
# # rmse_df_ep = rmse_df_ep.append(rmse_df_7)
# # rmse_df_ep = rmse_df_ep.append(rmse_df_8)

# rmse_df_ep

In [ ]:
# # df_test.to_csv('../../../Results/'+mdl+'/Predictions/'+crop+'_'+mdl+'_'+str(tst_year)+'.csv')

# rmse_df.to_csv('../../Results/3dcnn/compiled/lr_00001_'+crop+'_'+mdl+'_'+str(tst_year)+'.csv')

# rmse_df_ep.to_csv('../../Results/3dcnn/ep/lr_00001_'+crop+'_'+mdl+'_'+str(tst_year)+'.csv')

# print('csv created for lr_00001_'+crop+'_'+mdl)

In [ ]:
# rmse_df = pd.DataFrame({'loss_1': min(losses1), 'loss_2': min(losses2)}, index = ['modis_us_soy_ms_2019(16yrs)'])
# rmse_df

In [ ]:
# df_test.to_csv('../../../../../Results/18_aug/Predictions/modis_us_soy_ms_2019(16yrs).csv')
# rmse_df.to_csv('../../../../../Results/18_aug/error/modis_us_soy_ms_2019(16yrs).csv')

# print('file created modis_us_soy_ms_2019(16yrs)')

In [ ]:
# import IPython

# IPython.Application.instance().kernel.do_shutdown(True)

# print('kernal restarted')

In [ ]:


torch.cuda.empty_cache()
